# A3Data - Residência Porto Digital

## Processamento de Dados em Larga Escala - Aula 03


---
### Setup

In [ ]:
%%bash
# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

In [ ]:
%%bash
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-gols.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-estatisticas-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-cartoes.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/Legenda.txt

In [4]:
parametros = {
    'header': True,
    'inferSchema': True,
    'sep': ','
}

schema_full = """ID int, rodada int, data string, hora string, dia string, mandante string, visitante string, formacao_mandante string, 
  formacao_visitante string, tecnico_mandante string, tecnico_visitante string, vencedor string, arena string, mandante_placar string,
  visitante_placar string, mandante_estado string, visitante_estado string, estado_vencedor string"""

cartoes = spark.read.csv("campeonato-brasileiro-cartoes.csv", **parametros)
gols = spark.read.csv("campeonato-brasileiro-gols.csv", **parametros)
estats = spark.read.csv("campeonato-brasileiro-estatisticas-full.csv", **parametros)
full = spark.read.csv("campeonato-brasileiro-full.csv", **parametros, schema=schema_full)

cartoes.createOrReplaceTempView('cartoes')
gols.createOrReplaceTempView('gols')
estats.createOrReplaceTempView('estats')
full.createOrReplaceTempView('full')

# Dever de casa

Responda as perguntinhas:

1. Qual é a média de passes que o Cruzeiro fez nas partidas em que foi vencedor?
2. Quantos cartões amarelos o Grêmio levou no segundo tempo quando jogava em casa (mandante)?
3. Quantos gols o Flamengo marcou fora de casa (visitante) nas partidas em que tomou mais de um cartão amarelo?
4. Qual é a média de passes que o Cruzeiro fez nas partidas em que foi vencedor e marcou PELO MENOS 1 gol no primeiro tempo? (PEGADINHAS)

In [29]:
q("""

  SELECT
    AVG(e.passes) as med_passes,
    MIN(e.passes) as min_passes,
    MAX(e.passes) as max_passes
  FROM estats e
  INNER JOIN full f
    ON e.partida_id = f.ID
  WHERE e.clube = 'Cruzeiro'
    AND f.vencedor = 'Cruzeiro'
    AND e.passes > 0

""")

+-----------------+----------+----------+
|med_passes       |min_passes|max_passes|
+-----------------+----------+----------+
|436.9384615384615|229       |622       |
+-----------------+----------+----------+



Quantos cartões amarelos o Grêmio levou no segundo tempo quando jogava em casa (mandante)?

In [42]:
q("""
  -- COUNT(1) representa uma contagem simples de linhas na tabela
  SELECT
    COUNT(1) as qtd_cartoes_amarelos
  FROM cartoes c
  INNER JOIN full f
    ON c.partida_id = f.ID
  WHERE c.cartao = 'Amarelo'
    AND c.minuto > 45
    AND c.clube = 'Gremio'
    AND f.mandante = 'Gremio'

""")

+--------------------+
|qtd_cartoes_amarelos|
+--------------------+
|187                 |
+--------------------+



Quantos gols o Flamengo marcou fora de casa (visitante) nas partidas em que tomou mais de um cartão amarelo?

In [58]:
q("""

  SELECT
    COUNT(1) as contagem
  FROM (
    SELECT DISTINCT
      g.partida_id,
      g.clube as clube_gol,
      g.minuto as minuto_gol,
      f.visitante,
      c.clube as clube_cartao
    FROM gols g
    INNER JOIN full f
      ON g.partida_id = f.ID
    INNER JOIN cartoes c
      ON c.partida_id = g.partida_id
    WHERE g.clube = 'Flamengo'
      AND f.visitante = 'Flamengo'
      AND c.cartao = 'Amarelo'
      AND c.clube = 'Flamengo'
    ORDER BY g.partida_id
  )

""")

+--------+
|contagem|
+--------+
|181     |
+--------+



Qual é a média de passes que o Cruzeiro fez nas partidas em que foi vencedor e marcou PELO MENOS 1 gol no primeiro tempo?

In [ ]:
q("""

  SELECT *
  FROM gols
  WHERE clube = 'Cruzeiro'
    AND minuto LIKE '45%'
    OR minuto < 46
    

""", n=100)

In [78]:
q("""

  SELECT
    AVG(passes) as med_passes
  FROM (
    SELECT DISTINCT
      e.partida_id,
      e.clube,
      e.passes,
      f.vencedor
    FROM estats e
    INNER JOIN full f
      ON e.partida_id = f.ID
    INNER JOIN gols g
      ON g.partida_id = e.partida_id
    WHERE e.clube = 'Cruzeiro'
      AND e.passes > 0
      AND f.vencedor = 'Cruzeiro'
      AND g.clube = 'Cruzeiro'
      AND (g.minuto LIKE '45%' OR g.minuto < 46)
    ORDER BY partida_id
  )

""")

+-----------------+
|med_passes       |
+-----------------+
|428.0444444444444|
+-----------------+



## Algumas coisas mais

Se eu quiser produzir uma coluna baseada nos casos de outra coluna, posso usar o comando CASE WHEN (dentro do SELECT)

In [87]:
q("""
  -- Coluna se o gol foi no primeiro ou no segundo tempo
  SELECT
    partida_id,
    clube,
    minuto,
    CASE
      WHEN (minuto >= 46 OR minuto LIKE '90%')THEN 2
      ELSE 1 END AS tempo_gol,
    atleta
  FROM gols

""", n=100)

+----------+-------------+------+---------+-----------------------------------+
|partida_id|clube        |minuto|tempo_gol|atleta                             |
+----------+-------------+------+---------+-----------------------------------+
|4607      |Fluminense   |31    |1        |Rafael Sóbis                       |
|4607      |Fluminense   |45    |1        |Fred                               |
|4607      |Fluminense   |59    |2        |Nirley da Silva Fonseca            |
|4608      |Internacional|6     |1        |Charles Aránguiz                   |
|4610      |Athletico-PR |16    |1        |Dráusio Luis Salla Gil             |
|4611      |Sao Paulo    |13    |1        |Antônio Carlos dos Santos Aguiar   |
|4611      |Sao Paulo    |22    |1        |Douglas Pereira dos Santos         |
|4611      |Sao Paulo    |56    |2        |Luís Fabiano                       |
|4612      |Bahia        |80    |2        |Anderson Talisca                   |
|4612      |Cruzeiro     |63    |2      

In [92]:
q("""

  SELECT
    ID,
    mandante,
    CASE
      WHEN mandante = 'Atletico-MG' THEN 'Galo doido'
      WHEN mandante = 'Cruzeiro' THEN 'Cabuloso'
      WHEN mandante = 'Palmeiras' THEN 'Sem mundial'
      WHEN mandante = 'Flamengo' THEN 'Mengao'
      ELSE mandante 
    END AS apelido
  FROM full
  WHERE ID >= 5000

""", n = 200)

+----+-------------+-------------+
|ID  |mandante     |apelido      |
+----+-------------+-------------+
|5000|Figueirense  |Figueirense  |
|5001|Atletico-MG  |Galo doido   |
|5002|Flamengo     |Mengao       |
|5003|Santos       |Santos       |
|5004|Joinville    |Joinville    |
|5005|Ponte Preta  |Ponte Preta  |
|5006|Internacional|Internacional|
|5007|Sao Paulo    |Sao Paulo    |
|5008|Vasco        |Vasco        |
|5009|Gremio       |Gremio       |
|5010|Palmeiras    |Sem mundial  |
|5011|Chapecoense  |Chapecoense  |
|5012|Athletico-PR |Athletico-PR |
|5013|Fluminense   |Fluminense   |
|5014|Avai         |Avai         |
|5015|Cruzeiro     |Cabuloso     |
|5016|Sport        |Sport        |
|5017|Ponte Preta  |Ponte Preta  |
|5018|Coritiba     |Coritiba     |
|5019|Joinville    |Joinville    |
|5020|Santos       |Santos       |
|5021|Goias        |Goias        |
|5022|Atletico-MG  |Galo doido   |
|5023|Corinthians  |Corinthians  |
|5024|Internacional|Internacional|
|5025|Flamengo     |

Quando eu quero trocar o tipo de uma coluna, uso CAST

In [107]:
q("""

  SELECT 
    ID,
    data,
    hora,
    dia,
    CAST( CONCAT(data, ' ', hora) AS timestamp ) AS datahora
  FROM full

""")

+---+----------+-----+-------+-------------------+
|ID |data      |hora |dia    |datahora           |
+---+----------+-----+-------+-------------------+
|1  |2003-03-29|16:00|Sabado |2003-03-29 16:00:00|
|2  |2003-03-29|16:00|Sabado |2003-03-29 16:00:00|
|3  |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|4  |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|5  |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|6  |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|7  |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|8  |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|9  |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|10 |2003-03-30|16:00|Domingo|2003-03-30 16:00:00|
|11 |2003-03-30|18:00|Domingo|2003-03-30 18:00:00|
|12 |2003-03-30|18:00|Domingo|2003-03-30 18:00:00|
|13 |2003-04-05|16:00|Sabado |2003-04-05 16:00:00|
|14 |2003-04-05|16:00|Sabado |2003-04-05 16:00:00|
|15 |2003-04-05|16:00|Sabado |2003-04-05 16:00:00|
|16 |2003-04-05|18:00|Sabado |2003-04-05 18:00:00|
|17 |2003-04-06|16:00|Domingo|2

# DEVER DE CASA

1. Qual foi o time que mais venceu partidas desde 2014?
2. Qual foi o estado que teve mais derrotas entre 2008 e 2016?
3. Qual foi a média de cartões vermelhos que o Palmeiras tomou nas partidas em que jogou fora de casa, marcou pelo menos 1 gol no primeiro tempo?